<a href="https://colab.research.google.com/github/jcamp1234/is_job_binary_classification/blob/main/is_job_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import torchvision
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import pandas as pd
from google.colab import drive

In [15]:
from google.colab import files

uploaded = files.upload()


Saving data_fpf_v3_training.xlsx to data_fpf_v3_training.xlsx


In [16]:
import io
import pandas as pd
df = pd.read_excel(io.BytesIO(uploaded[list(uploaded.keys())[0]]))


In [17]:
def get_bert_embedding(text: str):
    # Load pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    # Tokenize input text and convert to tensor
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get the model's output
    outputs = model(**inputs)

    # Extract the last hidden state (embedding for each token)
    last_hidden_state = outputs.last_hidden_state

    # Compute the sentence embedding (mean pooling of token embeddings)
    sentence_embedding = last_hidden_state.mean(dim=1)  # Shape: [1, 768]

    return sentence_embedding

In [18]:
#Convert the data into an array
data = df.values.tolist()

In [ ]:
embeddings = []
for job in data:
  text = job[8]
  #embedding = get_bert_embedding(text)
  #embeddings.append([embedding, job[0]])

#print(embeddings[0][0].shape)
#print(embeddings[0][1])



In [ ]:
#print(embeddings[100][1])

# Arcitecture


In [25]:
# A model architecture to take a 768 dimensional embedding and return a binary classification

class BinaryClassifier(nn.Module):
  def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.lay1 = torch.nn.Linear(768, 500) # Fully connected layer
        self.lay2 = torch.nn.Linear(500, 250) # Fully connected layer
        self.lay3 = torch.nn.Linear(250, 100) # Fully connected layer
        self.lay4 = torch.nn.Linear(100, 25) # Fully connected layer
        self.lay5 = torch.nn.Linear(25, 1) # Fully connected layer
        self.relu = torch.nn.ReLU()
  def forward(self, x):
        x = self.relu(self.lay1(x))
        x = self.relu(self.lay2(x))
        x = self.relu(self.lay3(x))
        x = self.relu(self.lay4(x))
        x = self.lay5(x)
        return x

#Model
classifyer = BinaryClassifier()

#Criterion
criterion = torch.nn.BCEWithLogitsLoss()

#Optimizer
optimizer = torch.optim.Adam(classifyer.parameters(), lr=0.01)

In [26]:
# Use GPU for faster training (if available)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'We\'re using the {device}.')

classifyer = classifyer.to(device) # Put the model on the device

We're using the cpu.


In [27]:
#Training

from tqdm import tqdm

num_epochs = 10
for epoch in tqdm(range(num_epochs)):
  epoch_loss = 0

  for job in data:
    #Get the embedding of the text
    embedding = get_bert_embedding(job[1]).to(device)
    #print(embedding.shape)
    # Ensure the embedding requires gradients
    embedding = embedding.clone().detach().requires_grad_(True).to(device) #This line was added


    #Based on that embedding attempt to classify the text
    prediction = classifyer(embedding)

    #Get the label of the job
    label = job[0]
    target = torch.tensor([[label]], dtype=torch.float32).to(device)
    #print(target)

    # Calculate the loss
    loss = criterion(prediction, target)
    #print(loss)


    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(classifyer.parameters(), max_norm=1.0)
    optimizer.step()
    epoch_loss += loss.item()
  print(f'Epoch: {epoch}, Loss: {epoch_loss}')

 10%|█         | 1/10 [04:57<44:33, 297.08s/it]

Epoch: 0, Loss: 79353.2660767436


 20%|██        | 2/10 [09:49<39:15, 294.43s/it]

Epoch: 1, Loss: 134475.86131048203


 30%|███       | 3/10 [14:41<34:11, 293.04s/it]

...

Epoch: 7, Loss: 16811.132587730885


 90%|█████████ | 9/10 [44:08<04:52, 292.12s/it]

Epoch: 8, Loss: 40687.470483481884


100%|██████████| 10/10 [48:57<00:00, 293.79s/it]

Epoch: 9, Loss: 15525.440415501595


In [55]:
text1 = "Before converting to a gas heat stove I need to have my chimney cleaned.  Any suggestions for a reasonably priced local chimney cleaner?The small wood stove and stove pipe to be replaced with the gas stove is free to anyone who picks it up and takes it away. The outside dimensions of the box:  22x 13  The stove is 22 high.  Ideal for a small space.email or call 388-8024"
text2 = "I'm having difficulty uploading the latest version of Adobe Flashplayer and would also appreciate some guidance on how to upload my photos more  efficiently. I'm willing to pay someone for maybe an hour of their time."
embedding1 = get_bert_embedding(text1)
embedding2 = get_bert_embedding(text2)
text3 = "Looking for someone reliable to hay 18 +/- acres of excellent forage.  Must be small bales, not rounds or large squares.  Call Peggy 802-373-6708"
embedding3 = get_bert_embedding(text3)

print(classifyer(embedding1))
print(classifyer(embedding2))
print(classifyer(embedding3))


tensor([[-47.7877]], grad_fn=<AddmmBackward0>)
tensor([[-42.2166]], grad_fn=<AddmmBackward0>)
tensor([[-26.3001]], grad_fn=<AddmmBackward0>)


In [61]:
text5 = "I am looking to work 20-25 hours a week to help with your special needs. I have experience of over 8 years of providing personal care, respite, working with clients who have mobilitiy issues.I love to cook all kinds of meals. (I have had many compliments). I am available for running errands, shopping, housekeeping, laundry and whatever his or her needs may be.I am very dedicated to my work and excellent at multi-tasking and organizing. You will get 100% cooperation of me and my work ethic for I know that your time is valuable to you and your family. Looking forward to work one on one .You may contact me at 802-349-5370."
embedding3 = get_bert_embedding(text5)
text4 = "Your mom is dumb"
embedding4 = get_bert_embedding(text4)

print(classifyer(embedding3))
print(classifyer(embedding4))

tensor([[-32.0708]], grad_fn=<AddmmBackward0>)
tensor([[-38.2901]], grad_fn=<AddmmBackward0>)


To export the model

In [49]:
!pip install torch onnx onnxruntime

In [64]:
torch.save(classifyer.state_dict(), 'model_state_dict.pth')

In [50]:
# Load your model here
model = BinaryClassifier(*args, **kwargs)
model.load_state_dict(torch.load('your_model_path.pth'))
model.eval() # Set the model to evaluation mode

NameError: name 'args' is not defined

In [ ]:
dummy_input = torch.randn(1, input_size) # Replace input_size with the actual input size

In [ ]:
torch.onnx.export(model, dummy_input, "your_model.onnx", verbose=True, input_names=['input'], output_names=['output'])